In [82]:
import pandas as pd
import yfinance as yf
import pandas_ta as ta
import numpy as np
from backtesting import Backtest, Strategy

## Using the past 10 Year Historical Nifty50 Data

In [83]:
df = pd.DataFrame(yf.download("^NSEI", start="2014-07-01", end="2024-07-01"))

[*********************100%***********************]  1 of 1 completed


In [84]:
df.tail()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2024-06-24,23382.300781,23558.099609,23350.000000,23537.849609,23537.849609,239400
2024-06-25,23577.099609,23754.150391,23562.050781,23721.300781,23721.300781,298100
2024-06-26,23723.099609,23889.900391,23670.449219,23868.800781,23868.800781,287800
2024-06-27,23881.550781,24087.449219,23805.400391,24044.500000,24044.500000,515200
2024-06-28,24085.900391,24174.000000,23985.800781,24010.599609,24010.599609,354800


In [85]:
df.isnull().sum().any()

False

RSI (Relative Strength Index) Strategy
--------------------------------------

### Strategy Overview:

The **RSI (Relative Strength Index) Strategy** is designed to identify overbought and oversold conditions in a market using the RSI indicator. The RSI is a momentum oscillator that measures the speed and change of price movements and fluctuates between 0 and 100. This strategy focuses on **buying** when the market is considered **oversold** and **selling** when it is **overbought**, which often indicates a potential reversal in price.

### RSI Indicator:

-   **RSI Calculation**: The RSI is calculated using a 14-period look-back window (default setting), although this period can be adjusted for different market conditions.
-   **RSI Levels**:
    -   **RSI < 30**: The market is considered **oversold**, potentially signaling a buying opportunity.
    -   **RSI > 70**: The market is considered **overbought**, potentially signaling a selling opportunity.

### Key Features:

1.  **Buy Signal**:

    -   The strategy opens a long position when the RSI crosses **30 from below**, indicating that the market is in oversold territory and a bullish reversal may follow.
2.  **Sell Signal**:

    -   The strategy closes the long position when the RSI crosses **70 from above**, indicating that the market is overbought and may reverse downward.
3.  **No Shorting**:

    -   This is a **long-only strategy**, meaning it does not engage in short-selling even when the market is overbought.

4.  **Stop Loss**

	- Keep the stop loss at 5%
	
### Parameters:

-   **RSI Period**: 14 periods (default).
-   **RSI Overbought Level**: 70 (default).
-   **RSI Oversold Level**: 30 (default).

### Trade Execution:

-   **Buy**: When the RSI crosses below 30.
-   **Sell**: When the RSI crosses above 70.

### Customization:

-   You can adjust the **RSI period** to suit different market conditions. For example:
    -   A **shorter period** (e.g., 7) will make the RSI more sensitive to price changes, capturing quicker signals in volatile markets.
    -   A **longer period** (e.g., 21) will smooth out the indicator, making it less sensitive and more appropriate for slower-moving markets.

- 	The **Lower and Upper Threshold*** fo 30 and 70 can be tweaked to find the best parameters.

### Advantages:

-   **Simplicity**: The RSI strategy is straightforward and easy to understand, making it a great starting point for traders.
-   **Momentum-based**: It captures potential price reversals by measuring momentum.
-   **Customizable**: RSI periods and levels can be adjusted to match different market environments.

### Example Logic:

1.  **Buy** when the RSI crosses below 30, indicating oversold conditions.
2.  **Sell** when the RSI crosses above 70, indicating overbought conditions.
## Example Chart

| Date       | Price | RSI  | Signal         |
|------------|-------|------|----------------|
| 01-Jan-24  | 100   | 65   | No Action      |
| 15-Jan-24  | 90    | 28   | **Buy Signal** |
| 30-Jan-24  | 105   | 75   | **Sell Signal**|

---

In [86]:
from backtesting.lib import crossover

In [87]:
def RSI_Indicator(data, n):
    gain = pd.Series(data).diff()
    loss = gain.copy()
    gain[gain < 0] = 0	#negative gains will be treated as 0
    loss[loss > 0] = 0 	#positive gains will be treated as 0
    rs = gain.ewm(n).mean() / loss.abs().ewm(n).mean()
    return 100 - 100 / (1 + rs)

In [88]:
class RSI(Strategy):
    lower_limit=30
    upper_limit=70
    length=14
    def init(self):
        self.rsi = self.I(RSI_Indicator, self.data.Close , self.length)
    def next(self):
        # Close position when RSI crosses above the upper limit (70)
        if crossover(self.upper_limit, self.rsi, ):
            if self.position:
                self.position.close()

        # Buy when RSI crosses below the lower limit (30)
        elif crossover( self.rsi, self.lower_limit):
            if not self.position:
                stop_loss_price = self.data.Close[-1] * 0.85  # Stop loss at 85% of the current price
                self.buy(sl=stop_loss_price)

In [89]:
bt = Backtest(df, RSI, commission=.002, cash=10000)
bt.run()
bt.plot(filename='./plots/RSI(30,70,14).py',plot_volume=False, plot_pl=True)

C:\Users\tejas\AppData\Local\Temp\ipykernel_19040\2947909750.py:1: UserWarning: Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).
  bt = Backtest(df, RSI, commission=.002, cash=10000)
c:\Users\tejas\anaconda3\Lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
c:\Users\tejas\anaconda3\Lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
c:\Users\tejas\anaconda3\Lib\site-packages\backtesting\_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
c:\Us

GridPlot(id='p8361', ...)

Let us view the results for different window lengths

In [90]:
custom_params = dict(lower_limit=30, upper_limit=70)

In [91]:
stats, heatmap = bt.optimize(
	length = range(10,150,5),
	maximize='# Trades',
	return_heatmap=True,
	**custom_params
)

  0%|          | 0/28 [00:00<?, ?it/s]

In [92]:
heatmap.fillna(0).sort_values(ascending=False)

length  lower_limit  upper_limit
10      30           70             7.0
15      30           70             6.0
20      30           70             5.0
25      30           70             2.0
30      30           70             2.0
40      30           70             2.0
45      30           70             2.0
60      30           70             1.0
55      30           70             1.0
50      30           70             1.0
35      30           70             1.0
105     30           70             0.0
140     30           70             0.0
135     30           70             0.0
130     30           70             0.0
125     30           70             0.0
120     30           70             0.0
115     30           70             0.0
110     30           70             0.0
80      30           70             0.0
100     30           70             0.0
95      30           70             0.0
90      30           70             0.0
85      30           70             0.0
75     

Naturally, the maximum trades are achieved at the lowest value of window length because this makes the indicator more sensitive.

In [93]:
custom_params = dict(length=10, upper_limit=70, lower_limit=30)

In [94]:
bt = Backtest(df, RSI, commission=.002, cash=10000)
output = bt.run(**custom_params)
bt.plot(filename='./plots/RSI(30,70,10).py',plot_volume=False, plot_pl=True)
print(output)

C:\Users\tejas\AppData\Local\Temp\ipykernel_19040\4090335733.py:1: UserWarning: Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).
  bt = Backtest(df, RSI, commission=.002, cash=10000)
c:\Users\tejas\anaconda3\Lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
c:\Users\tejas\anaconda3\Lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
c:\Users\tejas\anaconda3\Lib\site-packages\backtesting\_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
c:\Us

Start                     2014-07-01 00:00:00
End                       2024-06-28 00:00:00
Duration                   3650 days 00:00:00
Exposure Time [%]                   15.980432
Equity Final [$]                 11849.644235
Equity Peak [$]                  12093.293649
Return [%]                          18.496442
Buy & Hold Return [%]              214.493025
Return (Ann.) [%]                    1.758769
Volatility (Ann.) [%]                6.984441
Sharpe Ratio                         0.251812
Sortino Ratio                        0.372679
Calmar Ratio                         0.103815
Max. Drawdown [%]                  -16.941433
Avg. Drawdown [%]                   -4.087525
Max. Drawdown Duration     1844 days 00:00:00
Avg. Drawdown Duration      399 days 00:00:00
# Trades                                    7
Win Rate [%]                        71.428571
Best Trade [%]                      18.950356
Worst Trade [%]                    -15.520814
Avg. Trade [%]                    

### Now let us test the strategy for different lower and upper thresholds

In [95]:
custom_params = dict(lower_limit=20, upper_limit=80, length=20)
bt = Backtest(df, RSI, commission=.002, cash=10000)
output = bt.run(**custom_params)
bt.plot(filename='./plots/RSI(20,80,20).py',plot_volume=False, plot_pl=True)
print(output)

C:\Users\tejas\AppData\Local\Temp\ipykernel_19040\1563289814.py:2: UserWarning: Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).
  bt = Backtest(df, RSI, commission=.002, cash=10000)
c:\Users\tejas\anaconda3\Lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
c:\Users\tejas\anaconda3\Lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
c:\Users\tejas\anaconda3\Lib\site-packages\backtesting\_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
c:\Us

Start                     2014-07-01 00:00:00
End                       2024-06-28 00:00:00
Duration                   3650 days 00:00:00
Exposure Time [%]                   15.246637
Equity Final [$]                 18337.202775
Equity Peak [$]                  18522.853166
Return [%]                          83.372028
Buy & Hold Return [%]              214.493025
Return (Ann.) [%]                    6.427182
Volatility (Ann.) [%]                8.681221
Sharpe Ratio                         0.740355
Sortino Ratio                        1.187952
Calmar Ratio                         0.429511
Max. Drawdown [%]                  -14.963968
Avg. Drawdown [%]                   -2.082211
Max. Drawdown Duration     1016 days 00:00:00
Avg. Drawdown Duration       37 days 00:00:00
# Trades                                    2
Win Rate [%]                             50.0
Best Trade [%]                      119.10003
Worst Trade [%]                    -11.918998
Avg. Trade [%]                    

Here, the condition for lower_threshold=20 and upper_threshold=80 is very strong number of trades executed is only 1. So it seems that the conditions do no arise very often. We can perform optimization on the values of the window_length to find the conditions for the maximum number of trades.

Optimzing for achieving the maximum number of trades with (20,80) RSI.

In [96]:
custom_params = dict(lower_limit=20, upper_limit= 80)

In [97]:
stats, heatmap = bt.optimize(
	length = range(10,150,5),
	maximize='# Trades',
	return_heatmap=True,
	**custom_params
)

  0%|          | 0/28 [00:00<?, ?it/s]

In [98]:
heatmap.fillna(0,inplace=True)

In [99]:
heatmap

length  lower_limit  upper_limit
10      20           80             5.0
15      20           80             2.0
20      20           80             2.0
25      20           80             1.0
30      20           80             0.0
35      20           80             0.0
40      20           80             0.0
45      20           80             0.0
50      20           80             0.0
55      20           80             0.0
60      20           80             0.0
65      20           80             0.0
70      20           80             0.0
75      20           80             0.0
80      20           80             0.0
85      20           80             0.0
90      20           80             0.0
95      20           80             0.0
100     20           80             0.0
105     20           80             0.0
110     20           80             0.0
115     20           80             0.0
120     20           80             0.0
125     20           80             0.0
130    

So it is clear that reducing the window length makes the indicator more sensitive and gives us more opportunities. Evaluation of returns at window_length = 10:

In [100]:
custom_params = dict(lower_limit=20, upper_limit=80, length=10)
bt = Backtest(df, RSI, commission=.002, cash=100000)
output = bt.run(**custom_params)
bt.plot(filename='./plots/RSI(20,80,10).py',plot_volume=False, plot_pl=True)
print(output)

c:\Users\tejas\anaconda3\Lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
c:\Users\tejas\anaconda3\Lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
c:\Users\tejas\anaconda3\Lib\site-packages\backtesting\_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
c:\Users\tejas\anaconda3\Lib\site-packages\backtesting\_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_scroll' property; using the latest value
  fig = gridplot(


Start                     2014-07-01 00:00:00
End                       2024-06-28 00:00:00
Duration                   3650 days 00:00:00
Exposure Time [%]                   23.970648
Equity Final [$]                153606.742229
Equity Peak [$]                 154680.144572
Return [%]                          53.606742
Buy & Hold Return [%]              214.493025
Return (Ann.) [%]                    4.508155
Volatility (Ann.) [%]               10.242537
Sharpe Ratio                          0.44014
Sortino Ratio                        0.654043
Calmar Ratio                         0.120137
Max. Drawdown [%]                  -37.525232
Avg. Drawdown [%]                   -2.326935
Max. Drawdown Duration     1325 days 00:00:00
Avg. Drawdown Duration       65 days 00:00:00
# Trades                                    5
Win Rate [%]                             60.0
Best Trade [%]                      59.009666
Worst Trade [%]                    -17.052946
Avg. Trade [%]                    

Here the number of trades is too less, and the exposure time is also not that good. Moreover, the avg. trade duration is 288 days, which is not great either. So this strategy is not very desireable.

## Conclusion of RSI Strategy Experiment

### Overview:
The experiment tested an **RSI (Relative Strength Index) strategy** with various configurations to evaluate its performance on stock data. The primary objective was to determine how different RSI parameters, such as the RSI period, overbought level, and oversold level, affect the strategy's profitability and risk metrics.

### Default Parameters:
- **RSI Length**: 14
- **Overbought Level**: 70
- **Oversold Level**: 30

These default parameters were compared to custom configurations to identify optimal values based on the backtest results.

### Conclusion:

- **Key Takeaways**:
  - **Longer RSI Periods** (21 vs. 14) helped smooth out noise and better identify true trend reversals. This came at a trade-off in the form of fewer number of opportunities.
  - **Higher Overbought/Oversold Levels** (80/20) focused on capturing stronger trends, which increased the profitability and reduced exposure to unnecessary trades during choppy market conditions, but again giving us fewer opportunities.
  
- **Future Considerations**:
  - More optimization can be performed by testing additional RSI periods and levels.
  - Take-profit mechanisms could be incorporated to further protect against large drawdowns.
  - The stop loss mechanism can be enhanced 


In summary, the RSI strategy give higher win rates but huge draw downs for some cases. The RSI Indicator is good indicator for momentum, and the contrarian reversal stratgy has high success rates for higher lenght intervals.